# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data1 = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
data1

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data2 = pd.pivot_table(data1, index='ProductName',columns='CustomerID',values='Quantity').fillna(0)

In [6]:
data2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
df_euclidean = pd.DataFrame(1/(1+squareform(pdist(data2.T, 'euclidean'))),columns=data2.columns, index=data2.columns)
df_euclidean.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [26]:
top=df_euclidean[33].sort_values(ascending=False)[1:].head()
index = top.index.tolist()
top

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
filtered_df = data1.loc[[i for i in index]]
filtered_df.head()

Quantity
CustomerID ProductName                         
264        Apricots - Halves                  1
           Apricots Fresh                     1
           Bacardi Breezer - Tropical         1
           Bagel - Plain                      1
           Banana - Leaves                    1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
products = filtered_df.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False)
products.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
products_customer = products.merge(data1.loc[33], how='left', on='ProductName')
filtro = products_customer[(products_customer['Quantity_y'].isnull())]
#print(data1.loc[33].shape)
#print(products_customer.shape)
filtro.head()

,Quantity_x,Quantity_y
ProductName,,
Butter - Unsalted,3,NaN
Wine - Ej Gallo Sierra Valley,3,NaN
Soup - Campbells Bean Medley,3,NaN
Wine - Blue Nun Qualitatswein,3,NaN
Chicken - Soup Base,2,NaN


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [70]:
def generateRecommendation(df):
    dic = {}
    customers_id= list(set(data['CustomerID'].tolist()))
    for c in customers_id:
        top = df[c].sort_values(ascending=False)[1:].head()
        index = top.index.tolist()
        filtro = data1.loc[[i for i in index]]
        products = filtro.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False)
        products_customer = products.merge(data1.loc[c], how='left', on='ProductName')
        filtro2 = products_customer[(products_customer['Quantity_y'].isnull())].head()
        dic[c]=filtro2.index.tolist()
    return dic
dic1=generateRecommendation(df_euclidean)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [74]:
df = pd.DataFrame(dic1)
df.T.head()

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [73]:
df_cosine = pd.DataFrame(1/(1+squareform(pdist(data2.T, 'cosine'))),columns=data2.columns, index=data2.columns)
dic2=generateRecommendation(df_cosine)
df2 = pd.DataFrame(dic2)
df2.T.head()

,0,1,2,3,4
83973,Pastry - Choclate Baked,Flour - Pastry,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn
59399,Ezy Change Mophandle,Anchovy Paste - 56 G Tube,Garlic - Peeled,Broom - Corn,Water - Aquafina Vitamin
92168,Wiberg Super Cure,Tuna - Salad Premix,Sage - Ground,Apricots Fresh,Beef - Inside Round
49159,"Turnip - White, Organic",Peas - Frozen,Lamb - Ground,Beer - Blue,Beef - Ground Medium
18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Ecolab - Lime - A - Way 4/4 L,Milk - 2%
